In [7]:
!pip install beautifulsoup4 requests


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Initial page URL
url = "https://bidassist.com/all-tenders/active?utm_source=Google&utm_medium=cpc&utm_campaign=15355181231&utm_term=bidassist%20login&utm_content=Search&gad_source=1&gclid=CjwKCAjw4_K0BhBsEiwAfVVZ_wRLjXvEp74X0NCGn2Ndtx6AjP3CYPczP1YeiQ0cvXjNrvbGipe2uBoCMrcQAvD_BwE&filter=LOCATION_STRING:Madhya%20Pradesh%7CUttar%20Pradesh%7CBihar&filter=VALUE:150000000%7C&filter=CATEGORY:Civil%20And%20Construction&sort=RELEVANCE:DESC&pageNumber=0&pageSize=10&tenderType=ACTIVE&tenderEntity=TENDER&year=2024&removeUnavailableTenderAmountCards=false&removeUnavailableEmdCards=false"

# Function to get tender names, locations, closing dates, and amounts from a single page
def get_tenders_info(soup):
    tenders = []
    
    card_elements = soup.find_all('div', class_='block card clearfix')
    
    for card in card_elements:
        tender_element = card.find('a', class_="anchor-wrap m-t-0 more-width")
        location_element = card.find('span', class_="truncate relative tender-locations").find('span')
        date_element = card.find('div', class_='date-wrap').find('span', class_=['truncate textHeading', 'truncate textHeading text-light-orange'])
        amount_element = card.find('div', class_='semibold date-wrap amount').find('span', class_='truncate textHeading')
        
        tender_name = tender_element.get_text(strip=True) if tender_element else "N/A"
        location = location_element.get_text(strip=True) if location_element else "N/A"
        closing_date = date_element.get_text(strip=True) if date_element else "N/A"
        tender_amount = amount_element.get_text(strip=True) if amount_element else "N/A"
        
        tenders.append((tender_name, location, closing_date, tender_amount))
    
    return tenders

# Function to get the next page URL
def get_next_page_url(soup):
    next_page_element = soup.find('a', rel='next')
    if next_page_element and 'href' in next_page_element.attrs:
        return next_page_element['href']
    return None

# List to store all tenders information
all_tenders_info = []

# Loop to navigate through all pages
while url:
    print(f"Fetching tenders from: {url}")
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Get tenders info from the current page
    tenders_info = get_tenders_info(soup)
    all_tenders_info.extend(tenders_info)
    
    # Get the URL for the next page
    url = get_next_page_url(soup)

# Convert the list of tenders to a DataFrame
df = pd.DataFrame(all_tenders_info, columns=["Tender Name", "Location", "Closing Date", "Tender Amount"])

# Save the DataFrame to an Excel file
excel_file = 'C:/Users/DELL/Downloads/BidAssist Tender details.xlsx'
df.to_excel(excel_file, index=False)

print(f"Data has been successfully written to {excel_file}")


Fetching tenders from: https://bidassist.com/all-tenders/active?utm_source=Google&utm_medium=cpc&utm_campaign=15355181231&utm_term=bidassist%20login&utm_content=Search&gad_source=1&gclid=CjwKCAjw4_K0BhBsEiwAfVVZ_wRLjXvEp74X0NCGn2Ndtx6AjP3CYPczP1YeiQ0cvXjNrvbGipe2uBoCMrcQAvD_BwE&filter=LOCATION_STRING:Madhya%20Pradesh%7CUttar%20Pradesh%7CBihar&filter=VALUE:150000000%7C&filter=CATEGORY:Civil%20And%20Construction&sort=RELEVANCE:DESC&pageNumber=0&pageSize=10&tenderType=ACTIVE&tenderEntity=TENDER&year=2024&removeUnavailableTenderAmountCards=false&removeUnavailableEmdCards=false
Fetching tenders from: https://bidassist.com/all-tenders/active/tender-page-2?utm_source=Google&utm_medium=cpc&utm_campaign=15355181231&utm_term=bidassist%20login&utm_content=Search&gad_source=1&gclid=CjwKCAjw4_K0BhBsEiwAfVVZ_wRLjXvEp74X0NCGn2Ndtx6AjP3CYPczP1YeiQ0cvXjNrvbGipe2uBoCMrcQAvD_BwE&filter=LOCATION_STRING:Madhya%20Pradesh%7CUttar%20Pradesh%7CBihar&filter=VALUE:150000000%7C&filter=CATEGORY:Civil%20And%20Const